<a href="https://colab.research.google.com/github/surajinchal/Generating-sonnet/blob/master/SONNET_%20GENERATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTING LIBRARIES**

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import Regularizer
import tensorflow.keras.utils as ku 
from keras.callbacks import ModelCheckpoint
import numpy as np 
import tensorflow as tf
import sys
from gensim.parsing.preprocessing import remove_stopwords
import re

# **READING, TOKENIZING AND CLEANING DATA**

In [17]:
tokenizer = Tokenizer()
#Reading Data
data = open(r'/content/drive/My Drive/dataset.txt').read()

#Making sentences Lower case and spliting it 
sonnet_14 = data.lower().split("<eos>")

#Removing the un wanted terms from the data
for i in range(len(sonnet_14)):
 sonnet_14[i] = sonnet_14[i].replace("\n",'')
 sonnet_14[i] = sonnet_14[i].replace("<eos>",'')
 sonnet_14[i] = re.sub("[`~!@#$+%*:()'?-]", ' ',sonnet_14[i]) 
 sonnet_14[i] = remove_stopwords(sonnet_14[i])


# **GENERATE VALUES**

In [18]:
#Generating the values for the data
tokenizer.fit_on_texts(sonnet_14)
total_words = len(tokenizer.word_index) + 1
input_sequences = []
for line in sonnet_14:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_sequence = token_list[:i+1]
        input_sequences.append(n_sequence)

In [19]:
max_sequence = max([len(x) for x in input_sequences])#selecting the maximum input sequence length
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence, padding='pre'))#padding prior zeros to the input sequence

predictors, label = input_sequences[:,:-1],input_sequences[:,-1]#splitting the data into predictor and label
                                                                                  
label = ku.to_categorical(label, num_classes=total_words)#making the variables to be either 0 or 1

# **CREATING A MODEL AND ITS LAYERS**

In [20]:
model = Sequential() #Using sequential model
model.add(Embedding(total_words, 50, input_length=max_sequence-1))  # Your Embedding Layer
model.add(Bidirectional(LSTM(150, return_sequences=True)))  # An LSTM Layer
model.add(Dropout(0.2))  #(# A dropout layer)
model.add(LSTM(100))  #(# Another LSTM Layer)
model.add(Dense(total_words/2, activation='relu'))  # A Dense Layer including regularizers
model.add(Dense(total_words, activation='softmax'))  # A Dense Layer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # Pick a loss function and an optimizer)


# **TRAINING THE DATA**

In [ ]:
train=model.fit(predictors,label,epochs=100,verbose=1)#traing the model on 100 epochs

Epoch 1/100
8371/8371 [==============================] - 307s 37ms/step - loss: 7.1205 - accuracy: 0.0663
Epoch 2/100
8371/8371 [==============================] - 305s 36ms/step - loss: 6.6212 - accuracy: 0.0824
Epoch 3/100
8371/8371 [==============================] - 304s 36ms/step - loss: 6.2654 - accuracy: 0.0928
Epoch 4/100
8371/8371 [==============================] - 304s 36ms/step - loss: 5.9854 - accuracy: 0.1012
Epoch 5/100
8371/8371 [==============================] - 304s 36ms/step - loss: 5.7454 - accuracy: 0.1100
Epoch 6/100
8371/8371 [==============================] - 303s 36ms/step - loss: 5.5269 - accuracy: 0.1182
Epoch 7/100
8371/8371 [==============================] - 303s 36ms/step - loss: 5.3247 - accuracy: 0.1275
Epoch 8/100
8371/8371 [==============================] - 304s 36ms/step - loss: 5.1280 - accuracy: 0.1374
Epoch 9/100
8371/8371 [==============================] - 305s 36ms/step - loss: 4.9341 - accuracy: 0.1499
Epoch 10/100
8371/8371 [======================

# **SAVING THE TRAINED MODEL**

In [7]:
# serialize model to JSON
model_json = model.to_json()
with open("jack1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


# **LOADING TRAINED MODEL AND PREDICTION**

In [21]:
#to read the .json and HDF5 file from the drive
from tensorflow.keras.models import model_from_json
json_file = open('/content/drive/My Drive/jack1.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('/content/drive/My Drive/model.h5')
model = loaded_model
#Feeding the trained model to predict the words
def predict(seed_text , seed = 10):
    for i in range( seed ):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=
        max_sequence , padding='pre')
        predicted = model.predict_classes(token_list, verbose=0 )
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


#Giving seed text for generation of the sonnet
data = open(r'/content/drive/My Drive/dataset2.txt').read()
data = data.split()
sonnet_14 = []

for i in range(len(data)):  
  data[i] = re.sub("[,`~!@#$+%*:()'?-]", '',data[i]) 
  data[i] = remove_stopwords(data[i])
for i in range(len(data)):
  if(data[i]!='<eos>'and data[i]!='.'and data[i]!=''):
    sonnet_14.append(data[i])

sonnet_14=sonnet_14[:70]
count=0
for i in sonnet_14:
  count=count+1
  if count>1 and count%15==0:
    print('\n')
    count+=1
  print(predict(i))


let day s patient long heaven s gates white things slave
forget days flag heart gaze d fields frames fair wood fair
matters rest sacred men heath bewayles breast d length d laid
reigned pass thou shall soule thou straight love desire assay golden
oer d sad breath tell d arms youth rise thou wit
happy lift like gives d miser s disguise happy ofttimes gods
realms s number mightie design eastward round time worth thought vain
long s smoke ghost thee pure s safer undo doth child
ago lay n right o s years chubby woman sound heavy
talked eye s despair — largely ’ forms blind hadde hart
love s spacious ploughman toil s wedded spotless olympian guide o
let day s patient long heaven s gates white things slave
voices car d s caressed computation honour friends grace n d
low thy s tender behold forward friends hate ah hate d


ruled d got love die ye king art hope shone child
brief breathe thou st heel away night feast s passing tell
sessions spectral thy look succour wall love famine thy reach d